# Pose-Based Form Checker (RTSP Stream)

Uses **pre-recorded exercise videos** to generate reference animations.
Captures video from **RTSP stream** (from laptop OBS/FFmpeg) → live feedback via stick figures (blue = reference, green/red = user).
Processed video streamed to phone via Flask.

**Setup:**

1. **On laptop:** Start RTSP stream with **ultrafast preset** to reduce codec issues:
   ```bash
   ffmpeg -f dshow -i video="YOUR_WEBCAM_NAME" -c:v libx264 -preset ultrafast -tune zerolatency -profile:v baseline -pix_fmt yuv420p -maxrate 1000k -bufsize 2000k -g 30 -f rtsp rtsp://0.0.0.0:8554/webcam.sdp
   ```

2. **Update RTSP URL** in cell 2 if your laptop IP changes.

3. **Run on RPi or any device** to process the stream and display results.

**Troubleshooting codec errors:**
- Use `ultrafast` preset (reduces encoding complexity)
- Use `baseline` profile (most compatible)
- Lower bitrate (`1000k` instead of `1500k`)
- If still failing, install GStreamer: `sudo apt install -y gstreamer1.0-tools gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly gstreamer1.0-libav`

**Folder structure:**
```
project/
├── recordFromVideo.ipynb
└── exercises/
    ├── squat.mp4
    ├── pushup.mp4
    └── dumbbell_curls.mp4
```

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
import glob
from flask import Flask, Response
import threading

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Global frame for streaming
current_frame = None

# RTSP stream configuration
RTSP_URL = "rtsp://192.168.0.36:8554/webcam.sdp"

# Set environment variables for better RTSP performance
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = (
    "rtsp_transport;tcp|"      # force TCP (no UDP packet loss)
    "fflags;nobuffer|"         # reduce buffering
    "flags;low_delay|"         # low-delay decoding
    "max_delay;0"              # zero delay
)

## Helper: Calculate Angle

In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    bc = c - b
    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

## Extract Reference from Video (Instead of Recording Yourself)

In [3]:
def extract_reference_from_video(video_path: str, exercise_name: str, target_fps: float = 10.0):
    """
    Extracts pose landmarks from a pre-recorded video and saves as .pkl
    """
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        return

    pose = mp_pose.Pose(static_image_mode=False,
                        model_complexity=1,
                        enable_segmentation=False,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = max(1, int(video_fps / target_fps))
    references = []
    frame_count = 0

    print(f"Extracting reference from {video_path} @ ~{target_fps} FPS...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            if results.pose_landmarks:
                lm_list = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
                references.append(lm_list)

        frame_count += 1

    cap.release()
    pose.close()

    # Save
    pkl_path = f"{exercise_name}.pkl"
    with open(pkl_path, 'wb') as f:
        pickle.dump(references, f)

    print(f"Saved {len(references)} reference frames → {pkl_path}")


## Auto-Discover & Extract All Videos in `exercises/` Folder

In [4]:
def extract_all_references_from_folder(folder='exercises', target_fps=10.0):
    if not os.path.exists(folder):
        print(f"Folder '{folder}' not found!")
        return

    video_extensions = ['*.mp4', '*.avi', '*.mov', '*.mkv', '*.wmv']
    video_files = []
    for ext in video_extensions:
        video_files.extend(glob.glob(os.path.join(folder, ext)))

    if not video_files:
        print("No videos found in exercises/ folder.")
        return

    print(f"Found {len(video_files)} video(s). Extracting references...\n")
    for video_path in video_files:
        name = os.path.splitext(os.path.basename(video_path))[0]
        extract_reference_from_video(video_path, name, target_fps)
    print("\nAll references extracted!")

## Run This Once: Extract All References

In [5]:
# Run this cell to process all videos in exercises/
extract_all_references_from_folder()

Found 5 video(s). Extracting references...



Error in cpuinfo: prctl(PR_SVE_GET_VL) failed
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Extracting reference from exercises/squat.mp4 @ ~10.0 FPS...


W0000 00:00:1762457870.938940    1805 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762457871.054602    1805 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762457871.236986    1805 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
W0000 00:00:1762457871.236986    1805 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Saved 51 reference frames → squat.pkl
Extracting reference from exercises/push-up.mp4 @ ~10.0 FPS...


...

Saved 27 reference frames → romanian_deadlift.pkl
Extracting reference from exercises/plank.mp4 @ ~10.0 FPS...


W0000 00:00:1762457896.559797    1843 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762457896.630963    1843 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Saved 39 reference frames → plank.pkl
Extracting reference from exercises/deadlift.mp4 @ ~10.0 FPS...


W0000 00:00:1762457902.872248    1852 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762457902.979707    1852 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Saved 35 reference frames → deadlift.pkl

All references extracted!


## Perform Exercise (Live Feedback + Streaming)

In [6]:
def perform_exercise(exercise_name):
    global current_frame
    
    pkl_path = f"{exercise_name}.pkl"
    if not os.path.exists(pkl_path):
        print(f"Reference not found: {pkl_path}. Run extraction first!")
        return

    with open(pkl_path, 'rb') as f:
        reference_sequence = pickle.load(f)
    
    if not reference_sequence:
        print("Empty reference sequence.")
        return

    pose = mp_pose.Pose()
    
    # Try different capture methods
    print("Attempting to connect to RTSP stream...")
    
    # Method 1: Direct RTSP with minimal buffering
    cap = cv2.VideoCapture(RTSP_URL, cv2.CAP_FFMPEG)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
    
    if not cap.isOpened():
        print("Method 1 failed. Trying GStreamer pipeline...")
        # Method 2: GStreamer (if available)
        gst_pipeline = (
            f"rtspsrc location={RTSP_URL} latency=0 protocols=tcp ! "
            "rtph264depay ! h264parse ! avdec_h264 ! "
            "videoconvert ! video/x-raw,format=BGR ! appsink drop=1"
        )
        cap = cv2.VideoCapture(gst_pipeline, cv2.CAP_GSTREAMER)
    
    if not cap.isOpened():
        print(f"ERROR: Cannot open RTSP stream: {RTSP_URL}")
        print("Troubleshooting:")
        print("1. Make sure FFmpeg is running on your laptop")
        print("2. Try: ffmpeg -f dshow -i video=\"YOUR_WEBCAM\" -c:v libx264 -preset ultrafast -tune zerolatency -f rtsp rtsp://0.0.0.0:8554/webcam.sdp")
        print("3. Check firewall isn't blocking port 8554")
        return
    
    print("Connected to RTSP stream!")
    print(f"🎥 Stream available at: http://[YOUR-DEVICE-IP]:5001/video")
    print(f"   Open this URL in your phone's browser to view the workout!")
    
    # Try to set lower resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    ref_index = 0
    animation_fps = 10
    frame_delay = 1.0 / animation_fps
    last_ref_time = time.time()
    angle_threshold = 20
    
    pose_connections = mp_pose.POSE_CONNECTIONS
    
    print(f"Starting {exercise_name}. Match the BLUE stick figure.")

    frame_count = 0
    error_count = 0
    max_errors = 10
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            error_count += 1
            if error_count >= max_errors:
                print(f"Too many errors ({error_count}). Stopping.")
                break
            print(f"Frame dropped at #{frame_count} (error {error_count}/{max_errors}). Continuing...")
            time.sleep(0.1)
            continue
        
        # Reset error counter on successful read
        error_count = 0
        frame_count += 1
        
        # Only process every 2nd frame to reduce CPU load
        if frame_count % 2 != 0:
            continue
        
        height, width, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        
        black = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Draw reference (BLUE)
        ref_lm = reference_sequence[ref_index]
        for conn in pose_connections:
            start, end = ref_lm[conn[0]], ref_lm[conn[1]]
            if start[3] > 0.1 and end[3] > 0.1:
                cv2.line(black,
                         (int(start[0] * width), int(start[1] * height)),
                         (int(end[0] * width), int(end[1] * height)),
                         (255, 0, 0), 2)  # BGR Blue
        
        # Draw user (GREEN if correct, RED if wrong)
        if results.pose_landmarks:
            user_lm = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
            
            for conn in pose_connections:
                i1, i2 = conn
                r1, r2 = ref_lm[i1], ref_lm[i2]
                u1, u2 = user_lm[i1], user_lm[i2]
                
                if all(x[3] > 0.1 for x in [r1, r2, u1, u2]):
                    # Find parent joint for angle
                    parent_idx = None
                    for c in pose_connections:
                        if c[1] == i2 and c[0] != i1:
                            parent_idx = c[0]; break
                        if c[0] == i2 and c[1] != i1:
                            parent_idx = c[1]; break
                    
                    if parent_idx is not None and ref_lm[parent_idx][3] > 0.1 and user_lm[parent_idx][3] > 0.1:
                        rp, up = ref_lm[parent_idx], user_lm[parent_idx]
                        
                        ref_angle = calculate_angle((r1[0], r1[1]), (r2[0], r2[1]), (rp[0], rp[1]))
                        user_angle = calculate_angle((u1[0], u1[1]), (u2[0], u2[1]), (up[0], up[1]))
                        
                        color = (0, 255, 0) if abs(ref_angle - user_angle) < angle_threshold else (0, 0, 255)
                        
                        cv2.line(black,
                                 (int(u1[0] * width), int(u1[1] * height)),
                                 (int(u2[0] * width), int(u2[1] * height)),
                                 color, 2)
        
        # Update global frame for Flask streaming (instead of cv2.imshow)
        current_frame = cv2.imencode('.jpg', black)[1].tobytes()
        
        # Advance reference
        if time.time() - last_ref_time >= frame_delay:
            last_ref_time = time.time()
            ref_index = (ref_index + 1) % len(reference_sequence)
        
        time.sleep(0.02)  # Small delay to prevent CPU overload
    
    cap.release()
    print(f"Exercise stopped. Processed {frame_count} frames.")


## Start Streaming Server (Run Once)

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return '''
    <html>
    <head>
        <title>Workout Form Checker</title>
        <style>
            body {
                margin: 0;
                padding: 20px;
                background: #1a1a1a;
                color: white;
                font-family: Arial, sans-serif;
                text-align: center;
            }
            h2 {
                color: #4CAF50;
                margin-bottom: 10px;
            }
            .info {
                background: #2d2d2d;
                padding: 15px;
                border-radius: 10px;
                margin-bottom: 20px;
                max-width: 800px;
                margin-left: auto;
                margin-right: auto;
            }
            .legend {
                display: flex;
                justify-content: center;
                gap: 30px;
                margin-bottom: 20px;
            }
            .legend-item {
                display: flex;
                align-items: center;
                gap: 10px;
            }
            .color-box {
                width: 30px;
                height: 20px;
                border: 2px solid white;
            }
            .blue { background: rgb(255, 0, 0); }  /* BGR in OpenCV */
            .green { background: rgb(0, 255, 0); }
            .red { background: rgb(0, 0, 255); }
            img {
                max-width: 100%;
                border: 3px solid #4CAF50;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0,0,0,0.5);
            }
        </style>
    </head>
    <body>
        <h2>🏋️ Workout Form Checker</h2>
        <div class="info">
            <div class="legend">
                <div class="legend-item">
                    <div class="color-box blue"></div>
                    <span>Reference (Target)</span>
                </div>
                <div class="legend-item">
                    <div class="color-box green"></div>
                    <span>Good Form</span>
                </div>
                <div class="legend-item">
                    <div class="color-box red"></div>
                    <span>Needs Adjustment</span>
                </div>
            </div>
            <p>Match the blue reference stick figure. Green = correct form, Red = adjust your position!</p>
        </div>
        <img src="/video" width="100%">
    </body>
    </html>
    '''

def gen():
    global current_frame
    while True:
        if current_frame is not None:
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + current_frame + b'\r\n')
        time.sleep(0.01)

@app.route('/video')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

def run_server():
    app.run(host='0.0.0.0', port=5001, threaded=True, use_reloader=False)

# Start in background
thread = threading.Thread(target=run_server, daemon=True)
thread.start()
print("🌐 Streaming server running on http://[YOUR-DEVICE-IP]:5001")
print("   Replace [YOUR-DEVICE-IP] with your Raspberry Pi's IP address")


🌐 Streaming server running on http://[YOUR-DEVICE-IP]:5001
   Replace [YOUR-DEVICE-IP] with your Raspberry Pi's IP address
 * Serving Flask app '__main__'
 * Debug mode: off
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.0.35:5001
Press CTRL+C to quit
Press CTRL+C to quit


## Perform Any Exercise

Just type the **exact filename (without extension)** from `exercises/` folder.

In [8]:
# Example: perform_exercise('squat')
# perform_exercise('pushup')
# perform_exercise('dumbbell_curls')

perform_exercise('romanian_deadlift')  # Change this to your video name

Attempting to connect to RTSP stream...


W0000 00:00:1762457909.495605    1860 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762457909.558394    1862 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Connected to RTSP stream!
🎥 Stream available at: http://[YOUR-DEVICE-IP]:5001/video
   Open this URL in your phone's browser to view the workout!
Starting romanian_deadlift. Match the BLUE stick figure.


[h264 @ 0x36ff4450] mmco: unref short failure
[h264 @ 0x364ab750] co located POCs unavailable
[h264 @ 0x364ab750] co located POCs unavailable
[h264 @ 0x367ce0d0] cabac decode of qscale diff failed at 50 58
[h264 @ 0x367ce0d0] error while decoding MB 50 58, bytestream -3
[h264 @ 0x364ab750] co located POCs unavailable
[h264 @ 0x37054da0] co located POCs unavailable
[h264 @ 0x364ab750] error while decoding MB 0 32, bytestream -5
[h264 @ 0x367ce0d0] cabac decode of qscale diff failed at 50 58
[h264 @ 0x367ce0d0] error while decoding MB 50 58, bytestream -3
[h264 @ 0x364ab750] co located POCs unavailable
[h264 @ 0x37054da0] co located POCs unavailable
[h264 @ 0x364ab750] error while decoding MB 0 32, bytestream -5
[h264 @ 0x364ab750] mmco: unref short failure
[h264 @ 0x37054da0] co located POCs unavailable
[h264 @ 0x364ab750] mmco: unref short failure
[h264 @ 0x37054da0] co located POCs unavailable
[h264 @ 0x364ab750] mmco: unref short failure
[h264 @ 0x364ab750] mmco: unref short failure


KeyboardInterrupt: 